In [241]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
from textblob import TextBlob

In [273]:
def ans(text,Q):
    # parse the Q with spacy parsing
    Q = nlp(Q)
    """
    for who questions, look for phrases with the NER tags: [PERSON || ORG] in the text that contains answer.
    only look for phrases where proper noun is the subject within sentence
    """
    if (str(Q[0]).lower() == "who"):
        # preprocess the Question
        cleanQ = list(Q)
        ## get rid of question mark at the end, if any
        cleanQ = [str(words) for words in cleanQ if str(words) != "?"]
        ## start from 1 to get rid of "who"
        ## fix the negation tokenization
        for i,token in enumerate(cleanQ):
            if token == "'s":
                cleanQ.remove(token)
                cleanQ[i - 1] = cleanQ[i - 1] + token
        strQbody = " ".join(cleanQ[i] for i in range(1, len(cleanQ)))
        
        text = nlp(text)
        ans = ""
        ent_dict = dict()
        for X in text.ents:
            ent_dict[X.text] = X.label_
        for token in text:
            for k,v in ent_dict.items():
                if (token.dep_ == "nsubj"):
                    if (v == "PERSON" or v == "ORG"):
                        ans = token.text + " " + strQbody +"."
    
    """
    for when questions, look for phrases with the NER tags: [DATE || TIME || CARDINAL] from the text that contains answer.
    """
    if (str(Q[0]).lower() == "when"):
        text = nlp(text)
        ans = ""
        ent_dict = dict()
        for X in text.ents:
            ent_dict[X.text] = X.label_
        for k,v in ent_dict.items():
            if (v == "DATE" or v == "CARDINAL" or v == "TIME"):
                ans = k
    """
    for where questions, look for phrases with the NER tags: GPE, LOC, in the text that contains answer.
    """
    if (str(Q[0]).lower() == "where"):
        text = nlp(text)
        ans = ""
        ent_dict = dict()
        for X in text.ents:
            ent_dict[X.text] = X.label_
        for k,v in ent_dict.items():
            if (v == "GPE" or v == "LOC"):
                ans = k
                    
                    
    """
    for why questions, extract words as "because, since, as 
    //ps: as can be very erroneous. need to find additional rule" 
    reconstruct sentense with prepositional phase to make the answer grammatically correct
    """
#     if (str(Q[0]).lower() == "because" or str(Q[0]).lower() == "since" or str(Q[0]).lower() == "as"):
        

    
    """
    for yes/no questions, do the sentiment analysis using Textblob
    """  
    if (str(Q[0]).lower() == "does" or str(Q[0]).lower() == "do" or str(Q[0]).lower() == "did"
        or str(Q[0]).lower() == "is" or str(Q[0]).lower() == "are" 
        or str(Q[0]).lower() == "have" or str(Q[0]).lower() == "has" or str(Q[0]).lower() == "had"):
            testimonial = TextBlob(text)
            print(testimonial.sentiment.polarity)
            if testimonial.sentiment.polarity >= 0:
                ans = "Yes"
            else:
                ans = "No"
    
    return ans

In [274]:
# Testing
## WHO
text = "Queen Hatshepsut concentrated on expanding Egypt's external trade by sending a commercial expedition to the land of Punt."
Q = "who concentrated on expanding Egypt's external trade?"

## WHEN

text = ('The Old Kingdom is the period in'
        'the third millennium (c. 2686-2181 BC)'
        'also known as the \'Age of the Pyramids\' or \'Age of the Pyramid Builders\''
        'as it includes the great 4th Dynasty when King Sneferu perfected the art of pyramid building'
        'and the pyramids of Giza were constructed under the kings Khufu, Khafre, and Menkaure.')
Q = "when is the Old Kingdom?"

text = "During the reign of Thutmose III (c. 1479–1425 BC), Pharaoh, originally referring to the king's palace, became a form of address for the person who was king"
Q = "when is the reign of Thutmose?"

text = ("The severity of the difficulties is indicated by the fact that the first known labor strike" 
        "in recorded history occurred during the 29th year of Ramesses III's reign, when the food rations for Egypt's favored"
        "and elite royal tomb-builders and artisans in the village of Deir el Medina could not be provisioned.")
Q = "when is the Ramesses's reign?"

ans(text,Q)

'the 29th year'

In [268]:
text = "During the reign of Thutmose III (c. 1479–1425 BC), Pharaoh, originally referring to the king's palace, became a form of address for the person who was king"
Q = "when is the reign of Thutmose?"
Q = nlp(Q)

In [269]:
if (str(Q[0]).lower() == "when"):
    text = nlp(text)
    ans = ""
    # dict to store NER tags of the text
    ent_dict = dict()
    for X in text.ents:
        ent_dict[X.text] = X.label_
    for k,v in ent_dict.items():
        if (v == "DATE" or v == "CARDINAL" or v == "TIME"):
            ans = k

In [270]:
ans

'1479–1425'

In [6]:
# testing
# text = nlp("Under King Djoser, the first king of the Third Dynasty of the Old Kingdom the royal capital of Egypt was moved to Memphis, where Djoser established his court.")
# Q = nlp("who is the first king of the Third Dynasty of the Old Kingdom?")
# Q = nlp("where does the Old Kingdom the royal capital of Egypt moved to?")

text = "The Old Kingdom is the period in the third millennium (c. 2686-2181 BC) also known as the \'Age of the Pyramids\' or \'Age of the Pyramid Builders\' as it includes the great 4th Dynasty when King Sneferu perfected the art of pyramid building and the pyramids of Giza were constructed under the kings Khufu, Khafre, and Menkaure. "
# Q = nlp("who perfected the art of pyramid building?")
Q = nlp("when is the Old Kingdom?")
# Q = nlp("Is the Old Kingdom also known as the \'Age of the Pyramids\' or \'Age of the Pyramid Builders\'?")




ans(text,Q)

'2686-2181'